In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import sys
import scipy.sparse as sparse
from scipy.sparse.linalg import spsolve
import random
from sklearn.preprocessing import MinMaxScaler
import implicit # The Cython library
import os
import pickle
import shelve

In [2]:
with open('./user_items.pkl', 'rb') as user_items:
        user_items = pickle.load(user_items)

In [3]:
with open('df_full.pkl', 'rb') as df_principal:
    df_full = pickle.load(df_principal)

In [4]:
knn = implicit.nearest_neighbours.ItemItemRecommender(40)

In [15]:
#df_full_tf = df_full.loc[:,["Product_Name"]].drop_duplicates()
#df_full_tf.dropna(inplace=True)
#df_full_tf["Product_Name_F"] = df_full["Product_Name"]+"_"+df_full["Familia"]
#df_full_tf.drop(columns=["Product_Name","Familia"],inplace=True)
#df_full_tf.head()

""
0
1
2
3
4


In [49]:
df_full_tf = pd.Series(df_full.Product_Name)
df_full_tf.iloc[0]

'Duraznos En Almibar En Mitades La Banda 850 Grs.'

In [28]:
#df_full_tf.reset_index(inplace=True)

In [50]:
df_full_tf.drop(columns="index", inplace=True)

In [52]:
df_full_tf = df_full_tf.drop_duplicates()

In [ ]:
#df_full.loc[df_full.index.isin([int(x[0]) for x in knn.similar_items(622)])]

In [ ]:
#sparse_item_user = sparse.csr_matrix(((df_full['SKU_ID_CODE'], df_full['SKU_ID_CODE'])))

In [53]:
pd.DataFrame(df_full_tf).sample(10)

,Product_Name
245,Acondicionador Plusbelle Detox Purificante Té ...
36778,Vino tinto blend reserva Ruca Malen 750 cc.
16768,Pañales Confort Sec Regular Pampers M 8 Un
497708,Postre Serenito Chocolate 100 gr
6493,Salchicha Vienissima Clasica 12Un 450 Grs.
507555,Agua Saborizada Aquarius Pera 600 Ml
500,Papel Higiénico Hoja Simple Elegante Aloe Vera...
259309,2x1 Baton extra leche 16 g
375102,Bananita Nestle DOLCA 14G X 15 Un
101745,Quitaesmalte Cutex Fortaceledor Calcio Y Vitam...


In [54]:
len(df_full_tf)

2610

In [63]:
#df_full_tf = pd.Series(df_full_tf.Product_Name_F)

In [55]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer=TfidfVectorizer();
df_full_tf_sparse=vectorizer.fit_transform(df_full_tf);

In [56]:
df_full_tf

0          Duraznos En Almibar En Mitades La Banda 850 Grs.
1                       Galletitas Frutigran Salvado 500 G.
2                   Galletitas Granix Granovita Limon 140 G
3           Hebras Cuatro Quesos Light Finlandia Chef 150gr
4                                Huevo color Avicoper 12 un
                                 ...                       
1027961           Protector Diario Kotex Ultra Flexible X50
1028230                 Panettone Bauducco Mini Choco 100Gr
1031136                           Galletitas Tita 5 X 18 G.
1040069                     Pandoro c/frutas Bauducco 500gr
1049077                    Silk Almendra Chocolate  190 cm3
Name: Product_Name, Length: 2610, dtype: object

In [57]:
pd.Series(vectorizer.vocabulary_)[:10]

duraznos       984
en            1008
almibar        397
mitades       1589
la            1379
banda          509
850            314
grs           1220
galletitas    1156
frutigran     1141
dtype: int64

In [58]:
from scipy.spatial.distance import pdist, squareform

In [59]:
dense_df_full_tf_sparse = df_full_tf_sparse.todense()

In [60]:
tfidf_sparse = dense_df_full_tf_sparse.transpose()

In [63]:
from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(1000)

In [64]:
svd_trans = svd.fit_transform(df_full_tf_sparse)

In [65]:
svd_trans.shape

(2610, 1000)

In [66]:
svd_trans_transp = svd_trans.transpose()

In [67]:
sqform = squareform(pdist(svd_trans_transp, metric="cosine"))

In [68]:
sqform

array([[0., 1., 1., ..., 1., 1., 1.],
       [1., 0., 1., ..., 1., 1., 1.],
       [1., 1., 0., ..., 1., 1., 1.],
       ...,
       [1., 1., 1., ..., 0., 1., 1.],
       [1., 1., 1., ..., 1., 0., 1.],
       [1., 1., 1., ..., 1., 1., 0.]])

In [69]:
knn2 = implicit.nearest_neighbours.ItemItemRecommender(40)

In [70]:
svd_trans

array([[ 0.11360929, -0.0971702 ,  0.0290829 , ...,  0.00352221,
         0.0076526 ,  0.0241237 ],
       [ 0.07577009, -0.04196328,  0.03941032, ..., -0.00332334,
         0.01413329, -0.00693726],
       [ 0.04181076, -0.04641696, -0.00347129, ...,  0.0194537 ,
        -0.01917865, -0.01060174],
       ...,
       [ 0.03302708, -0.03596601, -0.00023011, ...,  0.00204826,
         0.00096562,  0.00179764],
       [ 0.00488561, -0.00375971,  0.00019327, ...,  0.00283004,
         0.05545964, -0.00725247],
       [ 0.0771652 , -0.07393556, -0.03268533, ...,  0.00707295,
         0.00151324, -0.00209809]])

In [80]:
#knn2.fit(svd_trans)

In [72]:
knn.fit(df_full_tf_sparse)

In [73]:
knn.similar_items(100)

[(100, 1.0),
 (58, 0.8634060910337068),
 (727, 0.737690173970269),
 (2391, 0.7164894403534455),
 (52, 0.6340267202691946),
 (1232, 0.5866419512096139),
 (2362, 0.5821331765815105),
 (549, 0.5321071421919844),
 (51, 0.49311314653548877),
 (56, 0.4868430820656565)]

In [74]:
df_full_tf.iloc[100]

'Yogur Bebible Descremado Tregar Vainilla 900 G.'

In [78]:
df_full_tf.iloc[727]

'Yogur Bebible Descremado Milkaut Vainilla 1 L.'

In [59]:
df_full_tf_sparse

<1x1 sparse matrix of type '<class 'numpy.float64'>'
	with 1 stored elements in Compressed Sparse Row format>

In [79]:
df_full.loc[df_full.index.isin([int(x[0]) for x in knn.similar_items(100)])]

,SKU_ID,Order_ID,Ordered_Quantity,Product_Name,User ID,User_ID_CODE,SKU_ID_CODE
51,16276841,227424658,1,Papas Noisettes McCain 500 gr,15263554,44473,647
52,18977911,227424658,2,Salsa Filetto La Campagnola 340 G,15263554,44473,1637
56,15791674,227424690,5,Gaseosa Coca Cola sin azúcares 1.75 cc.,3037126,16009,191
58,17805066,227424690,1,Yerba Mate Con Palo Monte Esp. O.C. La Merced ...,3037126,16009,1268
100,17315137,227425887,1,Galletitas Crackers Granix Sandwich 3 U. X 200 G.,2040397,10977,1129
549,15717668,227431826,1,Lavandina Héroe común 1 l.,4704097,24417,153
727,15500909,227434586,1,Agua Mineral Sin Gas Glaciar Bajo Sodio 2 L.,16183314,46438,0
1232,16979862,227448049,1,Pan Bimbo Artesano 500 G.,356033,1504,1033
2362,16730499,227496923,1,Manteca Milkaut 200 G.,4770754,24756,819
2391,15527684,227497875,1,Papas Fritas Lay'S Clásicas 250 G.,3036278,16004,79
